# Issue Addressor Agent

This agent is responsible for addressing issues on the codebase. A title and description are extracted from the Gitlab or Github issue and passed to the agent.

It will start by distinguishing if the details are requests to change the code and if so, it will plan and execute the next steps.

In [1]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.types import Command
from rich import print  # noqa: A004

from automation.agents.issue_addressor import IssueAddressorAgent
from automation.agents.issue_addressor.conf import settings
from automation.utils import file_changes_namespace

source_repo_id = "srtab/daiv"
source_ref = "main"

config = {
    "run_name": "IssueAddressor",
    "recursion_limit": settings.RECURSION_LIMIT,
    "configurable": {"source_repo_id": source_repo_id, "source_ref": source_ref, "thread_id": "1"},
}
store = InMemoryStore()
checkpointer = InMemorySaver()
issue_addressor = IssueAddressorAgent(store=store, checkpointer=checkpointer)
issue_addressor_agent = await issue_addressor.agent

  + Exception Group Traceback (most recent call last):
  |   File "/home/sfr/work/personal/daiv/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3668, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/tmp/ipykernel_1422879/4097687435.py", line 21, in <module>
  |     issue_addressor_agent = await issue_addressor.agent
  |                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/sfr/work/personal/daiv/daiv/automation/agents/base.py", line 64, in agent
  |     return await self.compile()
  |            ^^^^^^^^^^^^^^^^^^^^
  |   File "/home/sfr/work/personal/daiv/daiv/automation/agents/issue_addressor/agent.py", line 60, in compile
  |     workflow.add_node("plan_and_execute", await self.plan_and_execute_subgraph(self.checkpointer, self.store))
  |                                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/home/sfr/work/personal/daiv/daiv/autom

## Graph 

In [ ]:
from IPython.display import Image, display  # noqa: A004

display(Image(issue_addressor.draw_mermaid()))

---

## Examples

Here are examples of the issue addressor agent in action. 

### Issue with no code changes

Before the agent is run, the issue is assessed to check if it is a request for code changes.

If it is not, the agent will not plan or execute any changes.

In [3]:
result = await issue_addressor_agent.ainvoke(
    {"issue_title": "Review README.md", "issue_description": "Is the README.md file up to date?"},
    config=config,
    subgraphs=True,
)

In [ ]:
print(result[1]["request_for_changes"])

### Issue with code changes

When the issue is a request for code changes, the agent will plan the changes to be made and wait for approval before executing them.

In [15]:
result = await issue_addressor_agent.ainvoke(
    {
        "issue_title": "Spelling mistakes",
        "issue_description": "There are some spelling mistakes in the README.md file. Please fix them.",
    },
    config=config,
)

In [ ]:
print(result)

Now that i have my plan defined, i need to approve it in order to execute it. The approval must be explicit, otherwise the agent will not execute the plan.

Here an example of a plan that is not explicitly approved:

In [ ]:
result = await issue_addressor_agent.ainvoke(Command(resume=[("human", "Hello DAIV")]), config=config, subgraphs=True)
print(result[1]["plan_approval_response"])

Now, an example of a plan that is explicitly approved:

In [5]:
result = await issue_addressor_agent.ainvoke(
    Command(resume=[("human", "I approve the plan")]), config=config, subgraphs=True
)

In [ ]:
print(result[1]["plan_approval_response"])

In [ ]:
async for item in store.asearch(file_changes_namespace(source_repo_id, source_ref)):
    print(f"============================= {item.key} =============================")
    print(item.value["data"].content)

### Issue with concrete code changes

There are cases where the issue description includes concrete code changes, like a code snippet, a code block, a list of choices, etc... and we need to ensure the agent will include the necessary context in the plan to perform the changes.

Let's see an example:

In [4]:
result = await issue_addressor_agent.ainvoke(
    {
        "issue_title": "New choices for model operations",
        "issue_description": """Add new choices for model operations:
* list - with translation "List"
* get - with translation "Get"
* search - with translation "Search"
* filter - with translation "Filter"
* bulk_create - with translation "Bulk create"
* bulk_update - with translation "Bulk update"
* bulk_delete - with translation "Bulk delete"
""",
    },
    config=config,
    subgraphs=True,
)

In [ ]:
print(result[1]["plan_goal"])
print(result[1]["plan_tasks"])

In [5]:
result = await issue_addressor_agent.ainvoke(
    Command(resume=[("human", "I approve the plan")]), config=config, subgraphs=True
)

In [ ]:
async for item in store.asearch(file_changes_namespace(source_repo_id, source_ref)):
    print(f"============================= {item.key} =============================")
    print(item.value["data"].content)

### Issue with code snippet

In [3]:
result = await issue_addressor_agent.ainvoke(
    {
        "issue_title": "New pre-commit config",
        "issue_description": """Add the following extra configurations to pre-commit config:

```json
  - repo: https://github.com/tox-dev/pyproject-fmt
    rev: "v2.5.1"
    hooks:
      - id: pyproject-fmt
```
""",
    },
    config=config,
    subgraphs=True,
)

In [ ]:
print(result[1]["plan_goal"])
print(result[1]["plan_tasks"])

In [5]:
result = await issue_addressor_agent.ainvoke(
    Command(resume=[("human", "I approve the plan")]), config=config, subgraphs=True
)

In [ ]:
print(result)